# Advanced Data Science Capstone

## Correlation of air pollution and Prevalence of Asthma bronchiale in Germany  

## Model definition: Logistic Regression in SparkML

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm


FeatureSetDenseMean = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.DenseMean.1.0.csv', index_col=None)
FeatureSetLongMean = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.LongMean.1.0.csv', index_col=None)

FeatureSetDensePerc50 = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.DensePerc50.1.0.csv', index_col=None)
FeatureSetDensePerc75 = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.DensePerc75.1.0.csv', index_col=None)

FeatureSetLongPerc50 = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.LongPerc50.1.0.csv', index_col=None)
FeatureSetLongPerc75 = pd.read_csv('Capstone.FeatureEng/Capstone.feature_eng.LongPerc75.1.0.csv', index_col=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

dfPolMeanLongDisease50perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease50perc.1.0.csv')
dfPolMeanLongDisease75perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease75perc.1.0.csv')
dfPolMeanLongDisease95perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolMeanLongDisease95perc.1.0.csv')

dfPolLongPerc75Disease50perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease50perc.1.0.csv')
dfPolLongPerc75Disease75perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease75perc.1.0.csv')
dfPolLongPerc75Disease95perc = spark.read.format("csv").load('Capstone.FeatureEng/Capstone.feature_eng.PolLongPerc75Disease95perc.1.0.csv')

df = dfPolLongPerc75Disease75perc
df.show()

ModuleNotFoundError: No module named 'pyspark'

In [3]:
from pyspark.ml.feature import StringIndexer


indexer = StringIndexer(inputCol="DiseaseRFeat", outputCol="classIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()
indexed.select('classIndex').distinct().show()

ModuleNotFoundError: No module named 'pyspark'

In [4]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer


encoder = OneHotEncoder(inputCol="classIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()

ModuleNotFoundError: No module named 'pyspark'

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["NO", "NO2", "PM1"],
                                  outputCol="features")

# For your special case that has string instead of doubles you should cast them first.
# expr = [col(c).cast("Double").alias(c) 
#         for c in vectorAssembler.getInputCols()]

# df2 = df2.select(*expr)
features_vectorized = vectorAssembler.transform(encoded)
features_vectorized.show()

ModuleNotFoundError: No module named 'pyspark'

In [6]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
l1NormData = normalizer.transform(features_vectorized)
l1NormData.show()

ModuleNotFoundError: No module named 'pyspark'

In [7]:
df_train = l1NormData.drop("CountyID").drop("DiseaseRFeat").drop("classIndex").drop("features").drop("NO").drop("NO2").drop("PM1")
df_train.show()

NameError: name 'l1NormData' is not defined

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer])
model = pipeline.fit(df)
prediction = model.transform(df)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
prediction.show()